In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re
import seaborn as sns

from bgs.load_bgs_amounts import load_bgs_amounts
from bgs.load_gilt_details import load_csv_blocks
from bgs.load_bgs_prices import load_prices
from bgs.gilt_analytics import gilt_yield

def clean_date(bgs_index):
    if re.match(r'^\d{2}\s\w{3}\s+\d{4}$', bgs_index):
        return pd.to_datetime(bgs_index, format="%d %b %Y", errors='coerce').to_period('M').to_timestamp('M')
    else:
        return pd.to_datetime(bgs_index).to_period('M').to_timestamp('M')

def clean_percentage(x):
    if x.strip() in ['Variable','Floating']:
        return x
    try:
        x = float(x)
    except (ValueError, TypeError):
        units, fractions = x.split(' ')
        num, denom = map(float, fractions.split('/'))
        x = float(units) + num / denom
        print(x)
    return x
tables = load_bgs_amounts("downloads/BGSAmounts.csv")

conv = tables['Conventionals']
conv.replace("Redeemed", "", inplace=True)
conv = conv.apply(pd.to_numeric, errors='coerce')
conv.fillna(0, inplace=True)
conv.index = pd.Index([clean_date(x) for x in list(conv.index)]).to_period('M').to_timestamp('M')

old = tables['Calculated indexed nominal Old-style']
old.replace("Redeemed", "", inplace=True)
old = old.apply(pd.to_numeric, errors='coerce')
old.fillna(0, inplace=True)
old.index = pd.Index([clean_date(x) for x in list(old.index)]).to_period('M').to_timestamp('M')

new = tables['Calculated indexed nominal New-style']
new.replace("Redeemed", "", inplace=True)
new = new.apply(pd.to_numeric, errors='coerce')
new.fillna(0, inplace=True)
new.index = pd.Index([clean_date(x) for x in list(new.index)]).to_period('M').to_timestamp('M')

new_no_idx = tables['Index-linked New-style']
new_no_idx.replace("Redeemed", "", inplace=True)
new_no_idx = new_no_idx.apply(pd.to_numeric, errors='coerce')
new_no_idx.fillna(0, inplace=True)
new_no_idx.index = pd.Index([clean_date(x) for x in list(new_no_idx.index)]).to_period('M').to_timestamp('M')

price_df=load_prices("downloads/BGSPrices.csv")
price_df.index = pd.to_datetime(price_df.index, format="%d %b %Y").to_period('M').to_timestamp('M')
for text in ['Amalgamated', 'Redeemed', 'redeemed']:
    price_df = price_df.replace(text, 0)
price_df = price_df.replace('missing', None)
price_df = price_df.ffill(axis=0)
price_df = price_df.fillna(0)
details = load_csv_blocks("downloads/BGSDetails.csv")

conv_details = details['Conventionals']
conv_details['%'] = conv_details['%'].apply(clean_percentage)

new_details = details['Index-Linked New-style']
new_details['%'] = new_details['%'].apply(clean_percentage)

old_details = details['Index-Linked Old-style']
old_details['%'] = old_details['%'].apply(clean_percentage)


In [ ]:
last = '2025-04-30'

In [ ]:
outstanding = conv.loc[last, conv.loc[last] !=0].reset_index()['index'].to_list()
outstanding[0]

In [ ]:
last_price = price_df.loc[last, [int(x) for x in outstanding]]
principal = conv.loc[last, outstanding]

In [ ]:
principal.index = principal.index.astype(int)
principal.name = 'balance_outstanding'
last_price.name = 'price'

In [ ]:
data_set_april = last_price.reset_index().merge(principal.reset_index(), on='index')
data_set_april.rename(columns={'index': 'Sequence'}, inplace=True)

In [ ]:
(data_set_april['balance_outstanding']*data_set_april['price']).sum()/data_set_april['balance_outstanding'].sum()

In [ ]:
(data_set_april['balance_outstanding']*0.16).sum()


In [ ]:
data_set_april.head()

In [ ]:
conv_details['Sequence'] = conv_details['Sequence'].astype(int)

In [ ]:
len([x for x in conv_details['Sequence'] if x in [int(y) for y in outstanding]])

In [ ]:
len(outstanding)

In [ ]:
conv_details.columns

In [ ]:
date_variables = ["Latest redemption date","Issue date","First coupon payable on date",]
static_variables = ['%', "Frequency","Sequence",'ISIN Code']+date_variables
data_set_april = data_set_april.merge(conv_details[static_variables].set_index("Sequence"), on='Sequence', how='inner')
for date_var in date_variables:
    data_set_april[date_var] = data_set_april[date_var].apply(clean_date)
data_set_april.head()

In [ ]:
(data_set_april['balance_outstanding']*data_set_april['%']).sum()/data_set_april['balance_outstanding'].sum()

In [ ]:
data_set_april['balance_outstanding'].plot(kind='hist')

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=data_set_april,
    x="Latest redemption date",
    y="price",
    size="balance_outstanding",
    legend=False,
    sizes=(10, 500)
)

# show the graph
plt.show()

In [ ]:
range = ax.get_xaxis().get_data_interval()


In [ ]:
(data_set_april['Latest redemption date'].min() - pd.Timestamp("1970-01-01")).days 

In [ ]:
X = ((data_set_april['Latest redemption date'] - pd.Timestamp("1970-01-01")).dt.days).to_list()
# X = [np.array(X)[:, np.newaxis]]
X = np.array(X).reshape(-1, 1)
y = data_set_april['price']

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.svm import SVR

pipeline = make_pipeline(
    preprocessing.StandardScaler(),
    SVR(kernel='linear', epsilon=0.01, C=100, gamma = 0.01),
)

In [ ]:
pipeline.fit(X, y)

In [ ]:
y_hat = pipeline.predict(X)

In [ ]:
sns.set_theme(style="darkgrid")

# use the scatterplot function to build the bubble map
ax = sns.scatterplot(
    data=data_set_april,
    x="Latest redemption date",
    y="price",
    size="balance_outstanding",
    legend=False,
    sizes=(10, 500)
)

# show the graph
ax.plot(data_set_april['Latest redemption date'], y_hat)
plt.show()

In [ ]:
decision_line = pd.DataFrame(
    {
        "Latest redemption date": data_set_april['Latest redemption date'],
        "decision_line_price": y_hat
    })

In [ ]:
data_set_april = data_set_april.merge(decision_line, on='Latest redemption date', how='inner')

In [ ]:
((data_set_april['decision_line_price'] - data_set_april['price'])/data_set_april['price']).plot(kind='hist', bins=50, title='Difference between decision line and actual price')

In [ ]:
(y_hat.max()-y_hat.min())/(range[1] -range[0])

In [ ]:
sns.set_theme(style="darkgrid")

f, axs = plt.subplots(1, 1, figsize=(8, 4))

condition = (data_set_april['decision_line_price'] - data_set_april['price']) > 2
data_set_april['segmentation'] = condition.apply(lambda x: 'low coupon' if x else 'high coupon')

# use the scatterplot function to build the bubble map
sns.scatterplot(
    data=data_set_april,
    x="Latest redemption date",
    y="price",
    size="balance_outstanding",
    sizes=(10, 500),
    hue='segmentation',
    legend='brief',
    ax=axs
)

# show the graph
axs.plot(data_set_april['Latest redemption date'], y_hat, )
sns.move_legend(axs, "upper left", bbox_to_anchor=(1, 1))
f.show()

In [ ]:
focus = data_set_april.loc[(data_set_april['segmentation'] == 'low coupon')&(data_set_april['price']<100)].copy()

In [ ]:
focus['balance_outstanding'].sum()

In [ ]:
((100 - focus['price'])*focus['balance_outstanding']).sum()/100

In [ ]:
focus

In [ ]:
# 

In [ ]:
data_set_april.tail(20)

In [ ]:
data_set_april[data_set_april['ISIN Code'].isin(['GB00BMBL1D50', 'GB00B54QLM75'])]

In [ ]:
convexity = price_df.loc["2020-05-31":, [32280, 32282]]

In [ ]:
convexity.head()

In [ ]:
"UKT_4%_2060", "UKT_0.5%_2061"

In [ ]:
convexity.loc["2020-05-31",32280]

In [ ]:
id = 32282
bond = data_set_april[data_set_april['Sequence']==id]
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]/100

today="2020-06-30"
clean_price=convexity.loc[today,id]


gilt_yield(
    today=today,
    issue_date=issue_date,
    maturity_date=maturity_date,
    first_cpn_date=first_cpn_date,
    last_cpn_date=last_cpn_date,
    clean_price=clean_price,
    coupon=coupon
)

In [ ]:
today, issue_date, maturity_date, first_cpn_date, last_cpn_date, clean_price, coupon

In [ ]:
r = convexity[32280].reset_index().iterrows().__next__()


In [ ]:
r[1]['index'].isoformat().split("T")[0]

In [ ]:
r[1][32280]

In [ ]:
def yield_series(price_series, issue_date, maturity_date, first_cpn_date, last_cpn_date, coupon):
    # Calculate the yield series based on the provided parameters
    name = price_series.name
    df = price_series.reset_index().rename(columns={name: "price"})
    df['yield'] = df.apply(lambda x: gilt_yield(
        today=x['index'].isoformat().split("T")[0],
        issue_date=issue_date,
        maturity_date=maturity_date,
        first_cpn_date=first_cpn_date,
        last_cpn_date=last_cpn_date,
        clean_price=x["price"],
        coupon=coupon/100
    ), axis=1)
    df.set_index('index', inplace=True)
    return df

In [ ]:
id = 32280
bond = data_set_april[data_set_april['Sequence']==id]
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]

ukt_4_2060 = yield_series(convexity[id], issue_date, maturity_date, first_cpn_date, last_cpn_date, coupon)


In [ ]:
id = 32282
bond = data_set_april[data_set_april['Sequence']==id]
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]
ukt_0_5_2061 = yield_series(convexity[id], issue_date, maturity_date, first_cpn_date, last_cpn_date, coupon)

In [ ]:
ukt_4_2060['norm_price'] = ukt_4_2060['price']/ukt_4_2060['price'].max()
ukt_0_5_2061['norm_price'] = ukt_0_5_2061['price']/ukt_0_5_2061['price'].max()

In [ ]:
f = plt.figure(figsize=(10, 5), )
fig, ax = plt.subplots(1, 1)
fig.suptitle("The cost of a low coupon bond")
sns.lineplot(data=ukt_4_2060, x='yield',y='norm_price', label='UKT 4% 2060', ax=ax)
sns.lineplot(data=ukt_0_5_2061, x='yield',y='norm_price', label='UKT 0.5% 2061', ax=ax)

In [ ]:
ukt_0_5_2061.tail()

In [ ]:
(ukt_4_2060['yield'] - ukt_0_5_2061['yield']).plot(title="Low Price Bonds no longer reflecting Economics")

In [ ]:
# 30 years
data_set_april[data_set_april['Sequence'].isin([32260, 32262])]

In [ ]:
thirty_years = price_df.loc["2020-06-30":, [32260, 32262]]

In [ ]:
id = 32260
bond = data_set_april[data_set_april['Sequence']==id]
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]

ukt4_25_2049 = yield_series(thirty_years[id], issue_date, maturity_date, first_cpn_date, last_cpn_date, coupon)

In [ ]:
id = 32262
bond = data_set_april[data_set_april['Sequence']==id]
issue_date=bond['Issue date'].dt.strftime("%Y-%m-%d").values[0]
maturity_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
first_cpn_date=bond['First coupon payable on date'].dt.strftime("%Y-%m-%d").values[0]
last_cpn_date=bond['Latest redemption date'].dt.strftime("%Y-%m-%d").values[0]
coupon=bond['%'].values[0]

ukt0_625_2050 = yield_series(thirty_years[id], issue_date, maturity_date, first_cpn_date, last_cpn_date, coupon)

In [ ]:
ukt4_25_2049['norm_price'] = ukt4_25_2049['price']/ukt4_25_2049['price'].max()
ukt0_625_2050['norm_price'] = ukt0_625_2050['price']/ukt0_625_2050['price'].max()

In [ ]:
f = plt.figure(figsize=(10, 5), )
fig, ax = plt.subplots(1, 1)
fig.suptitle("The cost of a low coupon bond")
sns.lineplot(data=ukt4_25_2049, x='yield',y='norm_price', label='UKT 4.25% 2049', ax=ax)
sns.lineplot(data=ukt0_625_2050, x='yield',y='norm_price', label='UKT 0.625% 2050', ax=ax)

In [ ]:
(ukt4_25_2049['yield'] - ukt0_625_2050['yield']).plot(title="Low Price Bonds no longer reflecting Economics")

In [ ]:
# 30 year UK Treasury Strip 07DEC2050C = 76800	UKT07DEC2050C	GB00B0BDTH75
strip = price_df.loc["2020-06-30":, 76800].reset_index().rename(columns={76800: "price"}).set_index('index')
f = plt.figure(figsize=(10, 5), )
fig, ax = plt.subplots(1, 1)
fig.suptitle("Strip versus low coupon")
sns.lineplot(data=ukt4_25_2049, x=ukt4_25_2049.index,y='price', label='UKT 4.25% 2049', ax=ax)
sns.lineplot(data=ukt0_625_2050, x=ukt0_625_2050.index,y='price', label='UKT 0.625% 2050', ax=ax)
sns.lineplot(data=strip, x=strip.index,y=strip['price'], label='Strip 07DEC2050C', ax=ax)


In [ ]:

ax = ukt4_25_2049['yield'].plot(title="The Tax Benefit of Low Coupon Bonds")
(ukt0_625_2050['price'] / strip['price']).plot(secondary_y=True)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
fig.set_figwidth(15)
fig.suptitle("The Tax Benefit of Low Coupon Bonds")
ax1.set_title("Rising Yields at the Maturity Date")
ax1.set_ylabel("Yield %")
ax2.set_title("Normal Behaviour in High Coupon")
ax3.set_title("Tax Benefit Raises Value of Low Coupon")
ax3.set_ylabel("Price Diff to Zero Coupon")
ax1 = ukt4_25_2049['yield'].plot(ax=ax1)
(ukt4_25_2049['price'] - strip['price']).plot(ax=ax2, secondary_y=True)
(ukt0_625_2050['price'] - strip['price']).plot(ax=ax3, secondary_y=True)

In [ ]:
boe_curves = "downloads/GLC Nominal month end data_2025 to present.xlsx"
df_curves = pd.ExcelFile(boe_curves)
df_curves.sheet_names

In [ ]:
df_curve = pd.read_excel(boe_curves, sheet_name='4. spot curve', skiprows=3)

In [ ]:
df_curve.head()

In [ ]:
df_curve = df_curve.set_index('years:')
spot_curve = df_curve.loc['2025-04-30'].to_dict()

In [ ]:
focus.head()

In [ ]:
flat_long = spot_curve.get(40)
focus['new coupon'] = ((focus['Latest redemption date'].dt.year-2025) + (focus['Latest redemption date'].dt.month)//6*0.5).apply(
    lambda x: spot_curve.get(x, flat_long))


In [ ]:
focus['refinanced'] = (1 - focus['price']/100)*focus['balance_outstanding']

In [ ]:
(focus['refinanced']*focus['new coupon']/100).sum()

In [ ]:
(focus['balance_outstanding']*focus['%']/100).sum()

### Bank of England Portfolio

In [ ]:
df_buy = pd.read_excel("downloads/gilt-purchase-operational-results.XLSX", sheet_name="APF Gilts", header=1)
df_sell = pd.read_excel("downloads/gilt-sales-time-series.XLSX", sheet_name="APF gilt sales", header=1)

df_boe = df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe['Operation date'] = pd.to_datetime(df_boe['Operation date'], format='%d-%m-%Y')
df_boe.set_index('Operation date', inplace=True)

df_boe_exit = df_sell[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe_exit['Operation date'] = pd.to_datetime(df_boe_exit['Operation date'], format='%d-%m-%Y')
df_boe_exit.set_index('Operation date', inplace=True)

In [ ]:
isin_bond_details = conv_details[['ISIN Code', 'Latest redemption date','%','Sequence']].copy()

gilt_level_buys = df_buy[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_buys.set_index('Operation date', inplace=True)
gilt_level_buys.index = pd.to_datetime(gilt_level_buys.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_buys.reset_index(inplace=True)

gilt_buys_ts = gilt_level_buys.pivot_table(
    index='Operation date', 
    columns='ISIN', 
    values='Total allocation (nominal £mn)', aggfunc='sum')

gilt_buys_ts = gilt_buys_ts.fillna(0).cumsum()

gilt_buys_total = gilt_level_buys.reset_index().groupby([ 'ISIN']).agg(
    Total_Allocation_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Allocation_Nominal=('Total allocation (nominal £mn)', 'sum'),
    Last_Purchase_Month=('Operation date', 'max')
)
gilt_buys_total.reset_index(inplace=True)

gilt_buys_total['maturity_date'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
gilt_buys_total['bgs_id'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Sequence'].values[0]
)

gilt_buys_total['maturity_date'] = pd.to_datetime(
    gilt_buys_total['maturity_date'], format="%d %b %Y"
)

In [ ]:
gilt_level_sales = df_sell[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_sales.set_index('Operation date', inplace=True)
gilt_level_sales.index = pd.to_datetime(gilt_level_sales.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_sales.reset_index(inplace=True)
gilt_level_sales.head()

In [ ]:
isin_bond_details = conv_details[['ISIN Code', 'Latest redemption date','%','Sequence']].copy()



In [ ]:
sales_by_isin=gilt_level_sales.groupby('ISIN')[['Total allocation (nominal £mn)', 'Total allocation (proceeds £mn)']].sum().reset_index()
sales_by_isin['maturity_date'] = sales_by_isin['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )
sales_by_isin['maturity_date'] = pd.to_datetime(
    sales_by_isin['maturity_date'], format="%d %b %Y"
)

In [ ]:
matured_sales = sales_by_isin['maturity_date'] > pd.Timestamp('2025-04-30')
matured = gilt_buys_total['maturity_date'] > pd.Timestamp('2025-04-30')

In [ ]:
data_set_april

In [ ]:
boe_portfolio = gilt_buys_total[matured].set_index('ISIN').merge(
    sales_by_isin[matured_sales][[
        'ISIN',
        'Total allocation (nominal £mn)',
        'Total allocation (proceeds £mn)'
        ]].set_index('ISIN'),
        how='left',
        left_index=True,
        right_index=True,
        suffixes=('_buy', '_sell')
        ).copy().fillna(0).reset_index()

boe_portfolio['mkt_price'] = boe_portfolio['bgs_id'].map(
    lambda x: data_set_april.loc[data_set_april['Sequence']==x]['price'].values[0])

In [ ]:
boe_portfolio.head()

In [ ]:
boe_low_coupon = [boe_isin for boe_isin in boe_portfolio['ISIN'].to_list() if boe_isin in focus['ISIN Code'].to_list()]
boe_high_coupon = [boe_isin for boe_isin in boe_portfolio['ISIN'].to_list() if boe_isin not in focus['ISIN Code'].to_list()]

In [ ]:
len(boe_low_coupon), len(boe_high_coupon)

In [ ]:
boe_portfolio['current position'] = boe_portfolio['Total_Allocation_Nominal'] - boe_portfolio['Total allocation (nominal £mn)']

In [ ]:
boe_portfolio[boe_portfolio['ISIN'].isin(boe_low_coupon)]['current position'].sum()

In [ ]:
boe_portfolio[boe_portfolio['ISIN'].isin(boe_high_coupon)]['current position'].sum()

In [ ]:
boe_portfolio['mtm'] = (boe_portfolio['mkt_price']-100) * boe_portfolio['current position']/100

In [ ]:
boe_portfolio[boe_portfolio['ISIN'].isin(boe_low_coupon)]['mtm'].sum()

In [ ]:
boe_portfolio[boe_portfolio['ISIN'].isin(boe_high_coupon)]['mtm'].sum()

In [ ]:
# To bring the portfolio to zero and swap everything up will cost
boe_portfolio[boe_portfolio['ISIN'].isin(boe_low_coupon)]['mtm'].sum() + boe_portfolio[boe_portfolio['ISIN'].isin(boe_high_coupon)]['mtm'].sum()

In [ ]:
# The old cost of interest was
(focus['balance_outstanding']*focus['%']/100).sum()

In [ ]:
# The new cost of interest is
(focus['refinanced']*focus['new coupon']/100).sum()

In [ ]:
# But the government has reduced the national debt by
((100 - focus['price'])*focus['balance_outstanding']).sum()/100

In [ ]:
# netting this together with the injection into the BoE portfolio the net reduction in national debt is
reduction = ((100 - focus['price'])*focus['balance_outstanding']).sum()/100 + boe_portfolio[boe_portfolio['ISIN'].isin(boe_low_coupon)]['mtm'].sum() + boe_portfolio[boe_portfolio['ISIN'].isin(boe_high_coupon)]['mtm'].sum()
reduction

In [ ]:
# reducing the national debt to gdp by percentage points
f"{reduction / (738159 *4):.2%}"